# LogWriter

In [14]:
from abc import ABC, abstractmethod
import json
from collections import deque
from typing import Any
import os
import datetime

In [15]:
class LogWriter(ABC):
    """LogWriter Abstract Class"""

    def __init__(self):
        self.queue = deque([])

    def append(self, log: Any) -> None:
        """Add the log on top of a stack."""
        self.queue.append(log)

    def flush(self) -> None:
        while self.queue:
            self.write(self.queue.popleft())

    @abstractmethod
    def write(
        self,
        log: Any,
    ) -> None:
        """Write logs"""


class jsonlogWriter(LogWriter):
    def write(self, log: Any) -> None:
        print(log)


class JSONLogWriter(LogWriter):
    def __init__(self, cwd_dir: str) -> None:
        super().__init__()
        self.dir = cwd_dir

        self._prepare_dir()

    def flush(self, filename: str | None = None):
        concat_log = {}
        while self.queue:
            log = self.queue.popleft()
            self._check_log(log)
            concat_log.update(log)
        self.write(concat_log, filename)

    def write(self, log: dict, filename: str | None = None) -> None:
        self._check_log(log)
        """Assuming the log is well formatted"""
        self._export_log(log, filename)

    def _export_log(self, log: dict, filename: str | None = None) -> None:
        """Export the logs to a JSON file.

        Args:
        ----
            file_path (str, optional): The file path to save the logs. If not provided, a default file path will be used.

        """
        if not filename:
            filename = self._default_filename()

        filepath = os.path.join(self.dir, filename)
        print(log, filepath)
        with open(filepath, "w") as file:
            json.dump(log, file, indent=4)

    def _default_filename(self) -> str:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        return f"log_{timestamp}.json"

    def _prepare_dir(self) -> None:
        os.makedirs(os.path.join(os.getcwd(), self.dir), exist_ok=True)

    def _check_log(self, log: dict) -> None:
        if not isinstance(log, dict):
            raise TypeError(
                f"The log should be a dict. It is received as a {type(log)}. Value= {log}"
            )

# Logwriter

In [4]:
import uuid


class LogGroup:
    def __init__(self, *, name: str):
        self.name = name
        self.id = uuid.uuid4()

# Decorator

In [14]:
from typing import Callable
import functools
import time
from datetime import datetime
import os
from log_writer import JSONLogWriter


def jsonlog(
    _func=None,
    *,
    writer: JSONLogWriter = None,
    flush: bool = True,
):
    if not writer:
        writer = JSONLogWriter(save_dir=os.getcwd())

    def jsonlog_decorator(func: Callable, writer: JSONLogWriter):
        @functools.wraps(func)
        def jsonlog_wrapper(*args, **kwargs):
            args_repr = [repr(a) for a in args]
            kwargs_repr = [f"{k}={repr(v)}" for k, v in kwargs.items()]
            signature = ", ".join(args_repr + kwargs_repr)

            uuidv1 = uuid.uuid1()

            timestamp = datetime.now()
            start_time = time.perf_counter()

            output = func(*args, **kwargs)

            end_time = time.perf_counter()
            run_time = end_time - start_time

            writer.append(
                log={
                    str(uuidv1): {
                        "name": func.__name__,
                        "args": args_repr,
                        "kwargs": kwargs_repr,
                        "signature": signature,
                        "output": repr(output),
                        "run_time": run_time,
                        "timestamp": str(timestamp),
                    }
                }
            )
            if flush:
                writer.flush(filename=writer._default_filename())

            return output

        return jsonlog_wrapper

    if _func is None:
        return lambda func: jsonlog_decorator(func, writer=writer)
    else:
        return jsonlog_decorator(_func, writer=writer)

In [8]:
writer1 = JSONLogWriter(save_dir="../logs")


@jsonlog(writer=writer1)
def f(x: int) -> int:
    return 2 * x


f(4)

8

In [12]:
f(9)

18

In [13]:
writer1.flush()

{'b28f4954-095d-11ef-8aee-294e27d4c8a5': {'name': 'f', 'args': ['4'], 'kwargs': [], 'signature': '4', 'run_time': 5.739966582041234e-07, 'timestamp': '2024-05-03 16:59:10.411831', 'output': '8'}, 'b42693b2-095d-11ef-8aee-294e27d4c8a5': {'name': 'f', 'args': ['9'], 'kwargs': [], 'signature': '9', 'run_time': 8.980023267213255e-07, 'timestamp': '2024-05-03 16:59:13.081054', 'output': '18'}, 'b9853e08-095d-11ef-8aee-294e27d4c8a5': {'name': 'f', 'args': ['9'], 'kwargs': [], 'signature': '9', 'run_time': 1.7120000848080963e-06, 'timestamp': '2024-05-03 16:59:22.090117', 'output': '18'}, 'b9853e09-095d-11ef-8aee-294e27d4c8a5': {'name': 'f', 'args': ['9'], 'kwargs': [], 'signature': '9', 'run_time': 1.143002009484917e-06, 'timestamp': '2024-05-03 16:59:23.178896', 'output': '18'}, 'baba0db2-095d-11ef-8aee-294e27d4c8a5': {'name': 'f', 'args': ['9'], 'kwargs': [], 'signature': '9', 'run_time': 1.3760000001639128e-06, 'timestamp': '2024-05-03 16:59:24.113959', 'output': '18'}} ../logs/log_202405

In [14]:
a = datetime.now()

In [17]:
str(a)

'2024-05-03 16:51:36.514765'